# 4.소득소비 데이터 전처리 

. 기준년, 분기, 상권코드 기준으로 주변 해당 상권 내 거주한 인구의 추정 소득금액  
. 지출 데이터는 해당 상권 내 상가를 분류(식료품, 생활용품 등)하고, 분류된 상가에서 발생한 카드 결제 금액을 합산하고 보정하여 계산  

In [1]:
import pandas as pd
from scipy.spatial import distance
import numpy as np

In [2]:
df = pd.read_csv('./data/서울시 우리마을가게 상권분석서비스(상권-소득소비).csv', encoding='cp949')

### 1. 2017년부터 2021년까지의 데이터를 가지고 있으므로 최종 데이터인 2021년 데이터 선택  

In [3]:
### 4.1. 2017년부터 2021년까지의 데이터를 가지고 있으므로 최종 데이터인 2021년 데이터 선택  
df_1 = df.loc[df['기준_년_코드']==2021]

In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6520 entries, 0 to 6519
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   기준_년_코드       6520 non-null   int64  
 1   기준_분기_코드      6520 non-null   int64  
 2   상권_구분_코드      6520 non-null   object 
 3   상권_구분_코드_명    6520 non-null   object 
 4   상권_코드         6520 non-null   int64  
 5   상권_코드_명       6520 non-null   object 
 6   월_평균_소득_금액    6480 non-null   float64
 7   소득_구간_코드      6480 non-null   float64
 8   지출_총금액        6388 non-null   float64
 9   식료품_지출_총금액    6388 non-null   float64
 10  의류_신발_지출_총금액  6388 non-null   float64
 11  생활용품_지출_총금액   6388 non-null   float64
 12  의료비_지출_총금액    6388 non-null   float64
 13  교통_지출_총금액     6388 non-null   float64
 14  여가_지출_총금액     6388 non-null   float64
 15  문화_지출_총금액     6388 non-null   float64
 16  교육_지출_총금액     6388 non-null   float64
 17  유흥_지출_총금액     6388 non-null   float64
dtypes: float64(12), int64(3), ob

In [5]:
df_1.columns[6:]

Index(['월_평균_소득_금액', '소득_구간_코드', '지출_총금액', '식료품_지출_총금액', '의류_신발_지출_총금액',
       '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액',
       '교육_지출_총금액', '유흥_지출_총금액'],
      dtype='object')

### 2. 결측치 확인 

. 데이터 확인 시 상권코드 기준으로 분기와 상관없이 데이터가 아예 존재하지 않음  
. 데이터 보간방법을 아래 두가지가 있음   
. 1. 데이터 삭제  
. 2. 지리적으로 가까운곳(유클리디안 거리로)의 값을 사용 

In [6]:
df[df['월_평균_소득_금액'].isnull()].sort_values(by = ['상권_코드', '기준_분기_코드'])

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
31247,2017,1,A,골목상권,2110310,길음역 10번,NaN,NaN,7680530.0,2157906.0,929290.0,485936.0,1019206.0,1267932.0,255508.0,240810.0,1008780.0,315162.0
31522,2017,1,A,골목상권,2110585,마포경찰서,NaN,NaN,355124249.0,84163827.0,41482163.0,23846320.0,40461022.0,62943987.0,12413333.0,10675759.0,65317271.0,13820567.0
16885,2019,2,A,골목상권,2110585,마포경찰서,NaN,NaN,199998259.0,45248746.0,23599180.0,13843089.0,22159464.0,36666065.0,7066558.0,5880357.0,37526239.0,8008561.0
29900,2017,2,A,골목상권,2110585,마포경찰서,NaN,NaN,163139030.0,39677915.0,19078310.0,10933538.0,19062630.0,28218214.0,5765732.0,5000894.0,29250365.0,6151432.0
15254,2019,3,A,골목상권,2110585,마포경찰서,NaN,NaN,199998259.0,45248746.0,23599180.0,13843089.0,22159464.0,36666065.0,7066558.0,5880357.0,37526239.0,8008561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606,2021,4,R,전통시장,2130308,청담삼익시장,NaN,NaN,81671028.0,18684612.0,11309429.0,6514972.0,10145059.0,15516256.0,5218093.0,2284347.0,8503220.0,3495040.0
8126,2020,4,R,전통시장,2130308,청담삼익시장,NaN,NaN,81671028.0,18684612.0,11309429.0,6514972.0,10145059.0,15516256.0,5218093.0,2284347.0,8503220.0,3495040.0
14646,2019,4,R,전통시장,2130308,청담삼익시장,NaN,NaN,81671028.0,18684612.0,11309429.0,6514972.0,10145059.0,15516256.0,5218093.0,2284347.0,8503220.0,3495040.0
21160,2018,4,R,전통시장,2130308,청담삼익시장,NaN,NaN,81671028.0,18684612.0,11309429.0,6514972.0,10145059.0,15516256.0,5218093.0,2284347.0,8503220.0,3495040.0


In [7]:
df.loc[df['지출_총금액'].isnull(), '상권_코드'].unique()

array([2110108, 2110648, 2110674, 2110704, 2110705, 2110764, 2110806,
       2111090, 2120060, 2120080, 2120098, 2120118, 2120218, 2130012,
       2130039, 2130046, 2130088, 2130153, 2130154, 2130155, 2130164,
       2130170, 2130176, 2130189, 2130196, 2130197, 2130205, 2130240,
       2130280, 2130284, 2130304, 2130310, 1001496, 2130085, 2130250,
       2130093, 2130184, 2110761, 2120023, 2120028, 2120029, 2120117,
       2120129, 2130005, 2130007, 2130013, 2130016, 2130017, 2130021,
       2130024, 2130048, 2130071, 2130079, 2130113, 2130121, 2130253,
       2130263, 2130292, 2110793, 2120025])

In [8]:
a = df['지출_총금액'].isnull()
b = df['상권_코드']==2130007

df[a&b]

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
30650,2017,2,R,전통시장,2130007,광장시장(광장전통시장),3050107.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32274,2017,1,R,전통시장,2130007,광장시장(광장전통시장),3693468.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# . 1. 데이터 삭제  
df_2_1 = df_1.dropna()

In [10]:
# . 2. 지리적으로 가까운곳(유클리디안 거리로)의 값을 사용 

In [11]:
# . 2.1 df_1에서 null값을 가지고 있는 데이터 셋 가져오기

# null값인 데이터 보관을 위한 데이터 셋 생성 
df_2_2_1 = pd.DataFrame()
# 컬럼별로 null값인 데이터 셋 저장 
for col in df_1.columns[6:] : 
    q = df_1[col].isnull()
    tmp = df_1[q]
    df_2_2_1 = pd.concat([df_2_2_1, tmp], axis = 0)
    
# 전체컬럼 기준 종복인컬럼 삭제 
df_2_2_1 = df_2_2_1.drop_duplicates() 

In [12]:
df_2_2_1.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
868,2021,4,A,골목상권,2110870,신림주공2단지(광신고등학교),NaN,NaN,525158961.0,182775990.0,56778514.0,35000230.0,89076459.0,59792839.0,16010293.0,17283761.0,49388632.0,19052243.0
1112,2021,4,D,발달상권,2120025,롯데백화점(시청광장 지하쇼핑센터),NaN,NaN,42776800.0,9753700.0,6378372.0,3414290.0,4949780.0,8313272.0,3229160.0,1234068.0,3606108.0,1898050.0
1222,2021,4,D,발달상권,2120137,문래역(문래로데오거리),NaN,NaN,63883830.0,12544623.0,8205981.0,5474138.0,6336884.0,11460297.0,3785327.0,1725862.0,11882007.0,2468711.0
1352,2021,4,R,전통시장,2130025,삼익패션타운(남대문시장),NaN,NaN,116229093.0,26285646.0,16688661.0,9929826.0,13192368.0,22183227.0,8157681.0,3433797.0,11270217.0,5087670.0
1425,2021,4,R,전통시장,2130113,신아타운,NaN,NaN,90288396.0,19005665.0,12176197.0,7310211.0,9891946.0,16641928.0,5750504.0,2467598.0,13379157.0,3665190.0


In [13]:
# . 2.2 기준정보데이터셋을 이용하여 거리가 가까운 데이터 찾기 
ma= pd.read_csv('./data/서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='cp949')

In [14]:
# . 2.2 df_2_2_1 의 상권코드를 기준으로 행별 x,y좌표값 입력 
df_2_2_2 = pd.merge(df_2_2_1, ma[['상권_코드', '엑스좌표_값', '와이좌표_값']], on = ['상권_코드'], how='left')

In [15]:
# . '엑스좌표_값', '와이좌표_값' 에 null이 존재하는지 확인
df_2_2_2[['엑스좌표_값', '와이좌표_값']].isnull().sum()

엑스좌표_값    0
와이좌표_값    0
dtype: int64

In [16]:
# 거리를 구할 데이터 셋의 경우 df_2_2_2의 상권코드를 가지고 있는 데이터 제거 
ma_notnull = ma.drop(ma[ma['상권_코드'].isin(df_2_2_1['상권_코드'].unique())].index)

In [17]:
# 기준정보에는 데이터가 존재하나 소득소비 엑셀에 데이터가 없는경우가 있어 추가 전처리 진행 
ma_notnull = ma_notnull[ma_notnull['상권_코드'].isin(df_1['상권_코드'].unique())]

In [18]:
ma_notnull.reset_index(inplace=True,drop=True)

In [23]:
ma_notnull[ma_notnull['상권_코드']==2110878]

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
86,202112,A,골목상권,2110878,미림여고,194062,440614,11620,11620745,NaN


In [ ]:
ma_notnull[ma_notnull['상권_코드']==2110878]

In [20]:
# 가까운 상권코드 컬럼 초기화
df_2_2_2['가까운_상권_코드'] = 0

In [21]:
# df_2_2_2 을 기준으로 ma_notnull에 상권들과 거리상 가까운 곳을 저장 

for i in range(len(df_2_2_2)):
# for i in range(1):
    eu_dis = np.inf
    clos_sc = np.inf
    null_xy = (df_2_2_2.loc[i, '엑스좌표_값'], df_2_2_2.loc[i, '와이좌표_값'])
    for j in range(len(ma_notnull)):
        notnull_xy = (ma_notnull.loc[j, '엑스좌표_값'], ma_notnull.loc[j, '와이좌표_값'])


        if distance.euclidean(null_xy, notnull_xy) < eu_dis :
            eu_dis = distance.euclidean(null_xy, notnull_xy)
            clos_sc = ma_notnull.loc[j, '상권_코드']
            
    print("best result for distance : ", eu_dis, "best result for sc : ",  clos_sc)
    df_2_2_2.loc[i, '가까운_상권_코드'] = clos_sc
    

best result for distance :  584.9111043568929 best result for sc :  2110878
best result for distance :  175.32826355154492 best result for sc :  1001492
best result for distance :  227.29716232280595 best result for sc :  2120140
best result for distance :  131.59027319676784 best result for sc :  2130024
best result for distance :  305.20321099228295 best result for sc :  2130116
best result for distance :  87.46427842267951 best result for sc :  2110431
best result for distance :  44.94441010848846 best result for sc :  2130253
best result for distance :  107.00467279516349 best result for sc :  2110884
best result for distance :  306.71974178392884 best result for sc :  2120167
best result for distance :  195.86219645454813 best result for sc :  2120216
best result for distance :  584.9111043568929 best result for sc :  2110878
best result for distance :  175.32826355154492 best result for sc :  1001492
best result for distance :  227.29716232280595 best result for sc :  2120140
bes

best result for distance :  527.4239281640529 best result for sc :  2110671
best result for distance :  288.47530223573733 best result for sc :  2110706
best result for distance :  204.26698215815497 best result for sc :  2110706
best result for distance :  273.99452549275503 best result for sc :  2120141
best result for distance :  259.8499567057882 best result for sc :  2110800
best result for distance :  642.3589339302443 best result for sc :  2111088
best result for distance :  363.6000550054964 best result for sc :  2110173
best result for distance :  754.0238723011361 best result for sc :  2110416
best result for distance :  575.127811881846 best result for sc :  2130183
best result for distance :  799.9031191338111 best result for sc :  2120119
best result for distance :  289.0069203323685 best result for sc :  2110991
best result for distance :  228.11619846034608 best result for sc :  2120015
best result for distance :  159.26079241294764 best result for sc :  1001493
best res

In [31]:
df_clos = df_1.copy()
df_clos.columns = ['가까운_기준_년_코드', '가까운_기준_분기_코드', '가까운_상권_구분_코드', '가까운_상권_구분_코드_명', '가까운_상권_코드', '가까운_상권_코드_명',
       '가까운_월_평균_소득_금액', '가까운_소득_구간_코드', '가까운_지출_총금액', '가까운_식료품_지출_총금액', '가까운_의류_신발_지출_총금액',
       '가까운_생활용품_지출_총금액', '가까운_의료비_지출_총금액', '가까운_교통_지출_총금액', '가까운_여가_지출_총금액', '가까운_문화_지출_총금액',
       '가까운_교육_지출_총금액', '가까운_유흥_지출_총금액']

In [34]:
# . 기존데이터 셋(df_2_2_1) 과 merge 하여 결측치 보간 
df_2_2_3 = pd.merge(df_2_2_2, df_clos, how='left'
        , left_on = ['기준_년_코드', '기준_분기_코드', '가까운_상권_코드'], right_on = ['가까운_기준_년_코드', '가까운_기준_분기_코드', '가까운_상권_코드'])

In [35]:
df_2_2_3.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,...,가까운_지출_총금액,가까운_식료품_지출_총금액,가까운_의류_신발_지출_총금액,가까운_생활용품_지출_총금액,가까운_의료비_지출_총금액,가까운_교통_지출_총금액,가까운_여가_지출_총금액,가까운_문화_지출_총금액,가까운_교육_지출_총금액,가까운_유흥_지출_총금액
0,2021,4,A,골목상권,2110870,신림주공2단지(광신고등학교),NaN,NaN,525158961.0,182775990.0,...,1.009296e+09,249519192.0,127104883.0,77649609.0,130056735.0,180714441.0,45217762.0,29220594.0,126416887.0,43395938.0
1,2021,4,D,발달상권,2120025,롯데백화점(시청광장 지하쇼핑센터),NaN,NaN,42776800.0,9753700.0,...,7.990605e+08,198543079.0,109338730.0,65537299.0,100540265.0,145153295.0,47266880.0,23816404.0,73392736.0,35471786.0
2,2021,4,D,발달상권,2120137,문래역(문래로데오거리),NaN,NaN,63883830.0,12544623.0,...,7.991680e+06,1866290.0,981223.0,653224.0,940436.0,1394935.0,408388.0,232166.0,1217100.0,297918.0
3,2021,4,R,전통시장,2130025,삼익패션타운(남대문시장),NaN,NaN,116229093.0,26285646.0,...,2.976414e+08,73259710.0,41017287.0,24731465.0,36698125.0,54843651.0,17461649.0,8976211.0,27166810.0,13486502.0
4,2021,4,R,전통시장,2130113,신아타운,NaN,NaN,90288396.0,19005665.0,...,1.936274e+08,41209289.0,26298345.0,15260766.0,21467594.0,35612871.0,12166321.0,5226380.0,28474651.0,7911146.0


In [42]:
# 컬럼 별 null값 입력 
df_2_2_4 = df_2_2_3.copy()

for i in df_2_2_4.columns[6:18]:
    col = i
    notnull_col = '가까운_'+i
    
    df_2_2_4[col] = np.where(df_2_2_4[col].isnull(), df_2_2_4[notnull_col], df_2_2_4[col])
    
    

In [48]:
df_2_2_4.iloc[:, :18].isnull().sum()

기준_년_코드         0
기준_분기_코드        0
상권_구분_코드        0
상권_구분_코드_명      0
상권_코드           0
상권_코드_명         0
월_평균_소득_금액      0
소득_구간_코드        0
지출_총금액          0
식료품_지출_총금액      0
의류_신발_지출_총금액    0
생활용품_지출_총금액     0
의료비_지출_총금액      0
교통_지출_총금액       0
여가_지출_총금액       0
문화_지출_총금액       0
교육_지출_총금액       0
유흥_지출_총금액       0
dtype: int64

In [53]:
df_2_2 = pd.concat([df_2_1, df_2_2_4], axis = 0).reset_index(drop = True)

In [54]:
df_2_2

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,...,가까운_지출_총금액,가까운_식료품_지출_총금액,가까운_의류_신발_지출_총금액,가까운_생활용품_지출_총금액,가까운_의료비_지출_총금액,가까운_교통_지출_총금액,가까운_여가_지출_총금액,가까운_문화_지출_총금액,가까운_교육_지출_총금액,가까운_유흥_지출_총금액
0,2021,4,A,골목상권,2110001,이북5도청사,4305934.0,8.0,5.204276e+08,129126028.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,4,A,골목상권,2110002,독립문역 1번,3463885.0,7.0,2.324652e+08,54998987.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,4,A,골목상권,2110003,세검정초등학교,3548020.0,7.0,1.349699e+09,336704220.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,4,A,골목상권,2110004,대신고등학교,2644953.0,6.0,7.127728e+08,205649728.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,4,A,골목상권,2110005,세검정,3703145.0,7.0,8.847016e+08,221446948.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6515,2021,1,R,전통시장,2130280,신림종합시장,1554706.0,3.0,3.079898e+09,854149885.0,...,3.079898e+09,854149885.0,377218935.0,241092660.0,398293664.0,489828980.0,108664775.0,103534755.0,371057366.0,136056721.0
6516,2021,1,R,전통시장,2130284,관악종합시장(신원시장),2656625.0,6.0,1.746199e+08,31559976.0,...,1.746199e+08,31559976.0,22539489.0,14712607.0,17105169.0,26404610.0,8416521.0,5644712.0,40479699.0,7757089.0
6517,2021,1,R,전통시장,2130304,신사상가,6105725.0,9.0,1.170393e+09,255344142.0,...,1.170393e+09,255344142.0,144885889.0,96324401.0,128681691.0,209264501.0,54262408.0,33378458.0,200618712.0,47632350.0
6518,2021,1,R,전통시장,2130310,강남개포시장,3919692.0,8.0,4.386259e+08,90538180.0,...,4.386259e+08,90538180.0,53407695.0,30295111.0,45911875.0,77061769.0,18038756.0,11265312.0,94336791.0,17770392.0


# 3. 데이터 내 seasonality 제거

 계절성을 없애고 기준년, 상권구분, 상권코드, 서비스 업종 기반으로 분석하기 위해 데이터 전처리 

'분기당_매출_금액' : 계절성을 없애기 위해 분기별 평균금액 사용  
'xxx건수' : 비율 확인을 위해 분기 합계 수치 계산 

In [50]:
df_2_1.shape

(6348, 18)

In [58]:
df_2_1.columns

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '월_평균_소득_금액', '소득_구간_코드', '지출_총금액', '식료품_지출_총금액', '의류_신발_지출_총금액',
       '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액',
       '교육_지출_총금액', '유흥_지출_총금액'],
      dtype='object')

In [55]:
6348+172

6520

In [56]:
df_2_2.shape

(6520, 38)

In [59]:
groupby_col = ['월_평균_소득_금액','지출_총금액', '식료품_지출_총금액', '의류_신발_지출_총금액',
       '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액',
       '교육_지출_총금액', '유흥_지출_총금액']

In [61]:
df_3_1 = df_2_1.groupby(['기준_년_코드','상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       ]).sum()[groupby_col].reset_index()

In [62]:
df_3_2 = df_2_2.groupby(['기준_년_코드','상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       ]).sum()[groupby_col].reset_index()

In [63]:
df_3_1.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,2021,A,골목상권,2110001,이북5도청사,17223736.0,2.081711e+09,5.165041e+08,259414528.0,148489468.0,263898408.0,353375044.0,88894600.0,58626788.0,304741364.0,87766228.0
1,2021,A,골목상권,2110002,독립문역 1번,13855540.0,9.298608e+08,2.199959e+08,115728360.0,68339440.0,113085488.0,163732316.0,40721844.0,26300048.0,143221132.0,38736244.0
2,2021,A,골목상권,2110003,세검정초등학교,14192080.0,5.398797e+09,1.346817e+09,664601184.0,394883832.0,682236096.0,909468360.0,218830432.0,156352120.0,796879056.0,228728960.0
3,2021,A,골목상권,2110004,대신고등학교,10579812.0,2.851091e+09,8.225989e+08,336680044.0,206673404.0,403269044.0,447374848.0,93626508.0,90460656.0,331460620.0,118947104.0
4,2021,A,골목상권,2110005,세검정,14812580.0,3.538806e+09,8.857878e+08,439529668.0,258953384.0,447955568.0,599291644.0,149914680.0,102427580.0,506177484.0,148768596.0


In [64]:
df_3_2.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,2021,A,골목상권,2110001,이북5도청사,17223736.0,2.081711e+09,5.165041e+08,259414528.0,148489468.0,263898408.0,353375044.0,88894600.0,58626788.0,304741364.0,87766228.0
1,2021,A,골목상권,2110002,독립문역 1번,13855540.0,9.298608e+08,2.199959e+08,115728360.0,68339440.0,113085488.0,163732316.0,40721844.0,26300048.0,143221132.0,38736244.0
2,2021,A,골목상권,2110003,세검정초등학교,14192080.0,5.398797e+09,1.346817e+09,664601184.0,394883832.0,682236096.0,909468360.0,218830432.0,156352120.0,796879056.0,228728960.0
3,2021,A,골목상권,2110004,대신고등학교,10579812.0,2.851091e+09,8.225989e+08,336680044.0,206673404.0,403269044.0,447374848.0,93626508.0,90460656.0,331460620.0,118947104.0
4,2021,A,골목상권,2110005,세검정,14812580.0,3.538806e+09,8.857878e+08,439529668.0,258953384.0,447955568.0,599291644.0,149914680.0,102427580.0,506177484.0,148768596.0


# 4. 파생변수 생성

. 분야별지출금액 대비 지출총금액 비율 변환  (ex > 식료품_지출_총금액/지출_총금액)  
    ['지출_총금액', '식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액',
       '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액']


In [65]:
df_4_1 = df_3_1.copy()
df_4_2 = df_3_2.copy()

In [66]:
df_4_1.columns

Index(['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '월_평균_소득_금액',
       '지출_총금액', '식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액',
       '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액'],
      dtype='object')

In [67]:
case_col = ['식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액',
       '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액']
rate_col = ['식료품_지출_총금액_비율', '의류_신발_지출_총금액_비율', '생활용품_지출_총금액_비율', '의료비_지출_총금액_비율',
       '교통_지출_총금액_비율', '여가_지출_총금액_비율', '문화_지출_총금액_비율', '교육_지출_총금액_비율', '유흥_지출_총금액_비율']

In [68]:
df_4_1[rate_col] = df_3_1[case_col].div(df_3_1['지출_총금액'], axis=0)
df_4_2[rate_col] = df_3_2[case_col].div(df_3_2['지출_총금액'], axis=0)

In [73]:
case_col = ['식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액',
       '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액']
rate_col = ['소득대비_식료품_지출_총금액_비율', '소득대비_의류_신발_지출_총금액_비율', '소득대비_생활용품_지출_총금액_비율', '소득대비_의료비_지출_총금액_비율',
       '소득대비_교통_지출_총금액_비율', '소득대비_여가_지출_총금액_비율', '소득대비_문화_지출_총금액_비율', '소득대비_교육_지출_총금액_비율', '소득대비_유흥_지출_총금액_비율']

In [74]:
df_4_1[rate_col] = df_3_1[case_col].div(df_3_1['월_평균_소득_금액'], axis=0)
df_4_2[rate_col] = df_3_2[case_col].div(df_3_2['월_평균_소득_금액'], axis=0)

In [75]:
df_4_2.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,...,유흥_지출_총금액_비율,소득대비_식료품_지출_총금액_비율,소득대비_의류_신발_지출_총금액_비율,소득대비_생활용품_지출_총금액_비율,소득대비_의료비_지출_총금액_비율,소득대비_교통_지출_총금액_비율,소득대비_여가_지출_총금액_비율,소득대비_문화_지출_총금액_비율,소득대비_교육_지출_총금액_비율,소득대비_유흥_지출_총금액_비율
0,2021,A,골목상권,2110001,이북5도청사,17223736.0,2.081711e+09,5.165041e+08,259414528.0,148489468.0,...,0.042161,29.987926,15.061455,8.621211,15.321787,20.516748,5.161168,3.403837,17.693105,5.095656
1,2021,A,골목상권,2110002,독립문역 1번,13855540.0,9.298608e+08,2.199959e+08,115728360.0,68339440.0,...,0.041658,15.877833,8.352497,4.932283,8.161752,11.817101,2.939030,1.898161,10.336741,2.795722
2,2021,A,골목상권,2110003,세검정초등학교,14192080.0,5.398797e+09,1.346817e+09,664601184.0,394883832.0,...,0.042367,94.899189,46.829019,27.824239,48.071607,64.082810,15.419194,11.016857,56.149561,16.116662
3,2021,A,골목상권,2110004,대신고등학교,10579812.0,2.851091e+09,8.225989e+08,336680044.0,206673404.0,...,0.041720,77.751751,31.822876,19.534695,38.116844,42.285709,8.849544,8.550308,31.329538,11.242837
4,2021,A,골목상권,2110005,세검정,14812580.0,3.538806e+09,8.857878e+08,439529668.0,258953384.0,...,0.042039,59.799697,29.672729,17.481991,30.241563,40.458289,10.120768,6.914905,34.172135,10.043395


In [76]:
df_4_1.to_csv('./data/4_1.소득소비_전처리완료.csv' , encoding='cp949')
df_4_2.to_csv('./data/4_2.소득소비_전처리완료.csv' , encoding='cp949')